##### IMPORTACION DE LIBRERIAS

In [1]:
import pandas as pd
import numpy as np

In [2]:
import setuptools

###### UTILIZACION DEL FRAMEWORK PYSPARK

In [3]:
import findspark
findspark.init()

In [4]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
from pyspark.sql import functions as F
from pyspark.sql.functions import col, sum, to_date
from pyspark.sql import Row

In [5]:
spark = SparkSession.builder \
        .appName("ETL-COVID") \
        .getOrCreate()

In [6]:
spark

### EXTRACCION DE DATOS MASIVOS

##### REGISTRO VACUNACION

###### DEFINICION DE SCHEMA

In [7]:
schema = StructType([
    StructField("FECHA_CORTE", StringType(), True),
    StructField("UUID", StringType(), True),
    StructField("GRUPO_RIESGO", StringType(), True),
    StructField("EDAD", IntegerType(), True),
    StructField("SEXO", StringType(), True),
    StructField("FECHA_VACUNACION", StringType(), True),
    StructField("DOSIS", IntegerType(), True),
    StructField("FABRICANTE", StringType(), True),
    StructField("DIRESA", StringType(), True),
    StructField("DEPARTAMENTO", StringType(), True),
    StructField("PROVINCIA", StringType(), True),
    StructField("DISTRITO", StringType(), True),
    StructField("CLASIFICACION_VACUNA", StringType(), True),
    StructField("TIPO_EDAD", StringType(), True)
])

###### LECTURA DEL ARCHIVO CSV

In [8]:
df_vacunacion = spark.read.csv("../DATASETS/vacunas_covid.csv", header=True, schema=schema)

###### NUMERO DE REGISTROS DEL DATAFRAME

In [28]:
num_vacunacion = df_vacunacion.count()
print(f"El numero de registros de vacunacion es: {num_vacunacion}")

El numero de registros de vacunacion es: 91611047


In [9]:
df_vacunacion.limit(100).toPandas()

,FECHA_CORTE,UUID,GRUPO_RIESGO,EDAD,SEXO,FECHA_VACUNACION,DOSIS,FABRICANTE,DIRESA,DEPARTAMENTO,PROVINCIA,DISTRITO,CLASIFICACION_VACUNA,TIPO_EDAD
0,20231210,32633894,PERSONAS DE 18 A 19 AÑOS,19,MASCULINO,20210911,1,PFIZER,TACNA,TACNA,TACNA,CORONEL GREGORIO ALBARRACIN LANCHIP,M,A
1,20231210,20142491,PERSONAS DE 5 A 7 AÑOS,6,MASCULINO,20221208,3,PFIZER,LIMA DIRIS ESTE,LIMA,LIMA,EL AGUSTINO,M,A
2,20231210,33351266,PERSONAS DE 5 A 7 AÑOS,6,MASCULINO,20220926,3,PFIZER,UCAYALI,UCAYALI,CORONEL PORTILLO,CALLERIA,M,A
3,20231210,30522825,PERSONAS DE 5 A 7 AÑOS,5,MASCULINO,20220417,2,PFIZER,CAJAMARCA,CAJAMARCA,CUTERVO,SANTO TOMAS,M,A
4,20231210,35534099,PERSONAS DE 5 A 7 AÑOS,5,MASCULINO,20220217,1,PFIZER,JUNIN,JUNIN,CHUPACA,CHUPACA,M,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,20231210,86014,"60 AÑOS A MÁS, ADULTO MAYOR",69,FEMENINO,20210808,1,SINOPHARM,UCAYALI,UCAYALI,CORONEL PORTILLO,CALLERIA,M,A
96,20231210,104239,"60 AÑOS A MÁS, ADULTO MAYOR",61,MASCULINO,20230901,5,MODERNA,UCAYALI,UCAYALI,CORONEL PORTILLO,CALLERIA,B,A
97,20231210,58879,PERSONAS DE 50 A 59 AÑOS,54,FEMENINO,20220912,4,MODERNA,UCAYALI,UCAYALI,CORONEL PORTILLO,CALLERIA,M,A
98,20231210,24153353,PERSONAS DE 50 A 59 AÑOS,55,MASCULINO,20221109,4,MODERNA,HUANUCO,HUANUCO,LEONCIO PRADO,RUPA-RUPA,M,A


##### CASOS POSITIVOS COVID

###### DEFINICION DE DTYPE

In [10]:
dtype_casos = {
    'UBIGEO': 'object',
    'id_persona': 'object'
}

###### LECTURA DEL ARCHIVO CSV

In [11]:
df_casos = pd.read_csv("../DATASETS/positivos_covid.csv", dtype = dtype_casos, sep = ";")

###### VERIFICACION DE TIPOS DE CAMPOS

In [14]:
df_casos.dtypes

FECHA_CORTE          int64
DEPARTAMENTO        object
PROVINCIA           object
DISTRITO            object
METODODX            object
EDAD               float64
SEXO                object
FECHA_RESULTADO    float64
UBIGEO              object
id_persona          object
dtype: object

###### NUMERO DE REGISTROS CASOS POSITIVOS

In [73]:
num_positivos = len(df_casos)
print(f"El numero de registros de casos positivos: {num_positivos}")

El numero de registros de casos positivos: 2256796


In [12]:
df_casos.head(10)

,FECHA_CORTE,DEPARTAMENTO,PROVINCIA,DISTRITO,METODODX,EDAD,SEXO,FECHA_RESULTADO,UBIGEO,id_persona
0,20211213,LIMA,LIMA,SAN MARTIN DE PORRES,PR,25.0,MASCULINO,20201217.0,150135,24662153
1,20211213,ICA,PISCO,PISCO,PR,20.0,FEMENINO,20200822.0,110501,24662175
2,20211213,HUANUCO,HUANUCO,HUANUCO,PR,22.0,FEMENINO,20200729.0,100101,24662197
3,20211213,ANCASH,SANTA,SANTA,AG,18.0,FEMENINO,20210630.0,021808,24662204
4,20211213,ANCASH,SANTA,NUEVO CHIMBOTE,AG,17.0,MASCULINO,20210404.0,021809,24662207
5,20211213,LIMA,LIMA,PACHACAMAC,PR,19.0,MASCULINO,20200708.0,150123,24662231
6,20211213,LIMA,LIMA,LA MOLINA,AG,21.0,FEMENINO,20210727.0,150114,24662254
7,20211213,APURIMAC,ANDAHUAYLAS,ANDAHUAYLAS,PCR,31.0,FEMENINO,20210130.0,030201,24662310
8,20211213,JUNIN,CHANCHAMAYO,CHANCHAMAYO,PR,35.0,FEMENINO,20210206.0,120301,24662314
9,20211213,PIURA,PIURA,CASTILLA,PR,34.0,MASCULINO,20200611.0,200104,24662316


In [12]:
df_casos.tail(10)

,FECHA_CORTE,DEPARTAMENTO,PROVINCIA,DISTRITO,METODODX,EDAD,SEXO,FECHA_RESULTADO,UBIGEO,id_persona
2256786,20211213,LIMA,LIMA,COMAS,PCR,35.0,MASCULINO,20210202.0,150110,NaN
2256787,20211213,LIMA,LIMA,COMAS,PR,31.0,MASCULINO,20200914.0,150110,NaN
2256788,20211213,LORETO,LORETO,NAUTA,PCR,31.0,MASCULINO,20200524.0,160301,NaN
2256789,20211213,ANCASH,SANTA,CHIMBOTE,PCR,6.0,MASCULINO,20210130.0,021801,NaN
2256790,20211213,LIMA,LIMA,SAN MARTIN DE PORRES,PCR,42.0,FEMENINO,20210216.0,150135,NaN
2256791,20211213,LIMA,HUAURA,HUACHO,PCR,46.0,MASCULINO,20210206.0,150801,NaN
2256792,20211213,LIMA,LIMA,SAN MARTIN DE PORRES,AG,34.0,MASCULINO,20210327.0,150135,NaN
2256793,20211213,LIMA,LIMA,SAN MARTIN DE PORRES,PCR,34.0,MASCULINO,20210405.0,150135,NaN
2256794,20211213,CALLAO,CALLAO,BELLAVISTA,PCR,28.0,MASCULINO,20210407.0,070102,NaN
2256795,20211213,CALLAO,CALLAO,BELLAVISTA,PCR,57.0,FEMENINO,20210408.0,070102,NaN


##### FALLECIDOS COVID

###### DEFINICION DE DTYPE

In [12]:
dtype_fallecidos = {
    'UBIGEO': 'object',
    'EDAD_DECLARADA': 'Int64',
    'id_persona': 'object'
}

###### LECTURA DEL ARCHIVO CSV

In [13]:
df_fallecidos = pd.read_csv("../DATASETS/fallecidos_covid.csv", dtype = dtype_fallecidos, sep = ";")

###### VERIFICACION DE TIPOS DE CAMPOS

In [20]:
df_fallecidos.dtypes

FECHA_CORTE             int64
FECHA_FALLECIMIENTO     int64
EDAD_DECLARADA          Int64
SEXO                   object
CLASIFICACION_DEF      object
DEPARTAMENTO           object
PROVINCIA              object
DISTRITO               object
UBIGEO                 object
id_persona             object
dtype: object

In [15]:
df_fallecidos.head(10)

,FECHA_CORTE,FECHA_FALLECIMIENTO,EDAD_DECLARADA,SEXO,CLASIFICACION_DEF,DEPARTAMENTO,PROVINCIA,DISTRITO,UBIGEO,id_persona
0,20231114,20210611,21,MASCULINO,Criterio SINADEF,LAMBAYEQUE,CHICLAYO,CHICLAYO,140101,24833991
1,20231114,20210317,45,MASCULINO,Criterio serolÃ³gico,PIURA,SULLANA,SULLANA,200601,24761117
2,20231114,20210602,62,FEMENINO,Criterio virolÃ³gico,ICA,PISCO,SAN CLEMENTE,110507,24767070
3,20231114,20210703,75,MASCULINO,Criterio virolÃ³gico,AREQUIPA,AREQUIPA,MIRAFLORES,040110,24751741
4,20231114,20210506,66,FEMENINO,Criterio virolÃ³gico,LIMA,LIMA,SAN JUAN DE LURIGANCHO,150132,24671820
5,20231114,20210321,39,MASCULINO,Criterio virolÃ³gico,LIMA,LIMA,SAN JUAN DE LURIGANCHO,150132,24709216
6,20231114,20210325,39,MASCULINO,Criterio virolÃ³gico,ANCASH,HUARMEY,HUARMEY,021101,24709228
7,20231114,20210706,31,FEMENINO,Criterio virolÃ³gico,LIMA,LIMA,LIMA,150101,25670721
8,20231114,20210530,75,FEMENINO,Criterio SINADEF,APURIMAC,CHINCHEROS,COCHARCAS,030603,25670904
9,20231114,20210320,57,MASCULINO,Criterio virolÃ³gico,LIMA,LIMA,RIMAC,150128,25671021


In [16]:
df_casos.tail(10)

,FECHA_CORTE,DEPARTAMENTO,PROVINCIA,DISTRITO,METODODX,EDAD,SEXO,FECHA_RESULTADO,UBIGEO,id_persona
2256786,20211213,LIMA,LIMA,COMAS,PCR,35.0,MASCULINO,20210202.0,150110,NaN
2256787,20211213,LIMA,LIMA,COMAS,PR,31.0,MASCULINO,20200914.0,150110,NaN
2256788,20211213,LORETO,LORETO,NAUTA,PCR,31.0,MASCULINO,20200524.0,160301,NaN
2256789,20211213,ANCASH,SANTA,CHIMBOTE,PCR,6.0,MASCULINO,20210130.0,021801,NaN
2256790,20211213,LIMA,LIMA,SAN MARTIN DE PORRES,PCR,42.0,FEMENINO,20210216.0,150135,NaN
2256791,20211213,LIMA,HUAURA,HUACHO,PCR,46.0,MASCULINO,20210206.0,150801,NaN
2256792,20211213,LIMA,LIMA,SAN MARTIN DE PORRES,AG,34.0,MASCULINO,20210327.0,150135,NaN
2256793,20211213,LIMA,LIMA,SAN MARTIN DE PORRES,PCR,34.0,MASCULINO,20210405.0,150135,NaN
2256794,20211213,CALLAO,CALLAO,BELLAVISTA,PCR,28.0,MASCULINO,20210407.0,070102,NaN
2256795,20211213,CALLAO,CALLAO,BELLAVISTA,PCR,57.0,FEMENINO,20210408.0,070102,NaN


### TRANSFORMACION DE DATOS MASIVOS:

##### FILTRO Y LIMPIEZA DE DATOS MASIVOS REGISTROS DE VACUNACION

###### DATAFRAME DF_VACUNACION ANTES DE LA LIMPIEZA Y FORMATEO

In [13]:
df_vacunacion.limit(100).toPandas()

,FECHA_CORTE,UUID,GRUPO_RIESGO,EDAD,SEXO,FECHA_VACUNACION,DOSIS,FABRICANTE,DIRESA,DEPARTAMENTO,PROVINCIA,DISTRITO,CLASIFICACION_VACUNA,TIPO_EDAD
0,20231210,32633894,PERSONAS DE 18 A 19 AÑOS,19,MASCULINO,20210911,1,PFIZER,TACNA,TACNA,TACNA,CORONEL GREGORIO ALBARRACIN LANCHIP,M,A
1,20231210,20142491,PERSONAS DE 5 A 7 AÑOS,6,MASCULINO,20221208,3,PFIZER,LIMA DIRIS ESTE,LIMA,LIMA,EL AGUSTINO,M,A
2,20231210,33351266,PERSONAS DE 5 A 7 AÑOS,6,MASCULINO,20220926,3,PFIZER,UCAYALI,UCAYALI,CORONEL PORTILLO,CALLERIA,M,A
3,20231210,30522825,PERSONAS DE 5 A 7 AÑOS,5,MASCULINO,20220417,2,PFIZER,CAJAMARCA,CAJAMARCA,CUTERVO,SANTO TOMAS,M,A
4,20231210,35534099,PERSONAS DE 5 A 7 AÑOS,5,MASCULINO,20220217,1,PFIZER,JUNIN,JUNIN,CHUPACA,CHUPACA,M,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,20231210,86014,"60 AÑOS A MÁS, ADULTO MAYOR",69,FEMENINO,20210808,1,SINOPHARM,UCAYALI,UCAYALI,CORONEL PORTILLO,CALLERIA,M,A
96,20231210,104239,"60 AÑOS A MÁS, ADULTO MAYOR",61,MASCULINO,20230901,5,MODERNA,UCAYALI,UCAYALI,CORONEL PORTILLO,CALLERIA,B,A
97,20231210,58879,PERSONAS DE 50 A 59 AÑOS,54,FEMENINO,20220912,4,MODERNA,UCAYALI,UCAYALI,CORONEL PORTILLO,CALLERIA,M,A
98,20231210,24153353,PERSONAS DE 50 A 59 AÑOS,55,MASCULINO,20221109,4,MODERNA,HUANUCO,HUANUCO,LEONCIO PRADO,RUPA-RUPA,M,A


###### VERIFICACION DE CAMPOS NULL

In [24]:
nulos_por_columna = df_vacunacion.select([sum(col(c).isNull().cast("int")).alias(c) for c in df_vacunacion.columns])

In [25]:
nulos_por_columna.show()

+-----------+----+------------+----+----+----------------+-----+----------+------+------------+---------+--------+--------------------+---------+
|FECHA_CORTE|UUID|GRUPO_RIESGO|EDAD|SEXO|FECHA_VACUNACION|DOSIS|FABRICANTE|DIRESA|DEPARTAMENTO|PROVINCIA|DISTRITO|CLASIFICACION_VACUNA|TIPO_EDAD|
+-----------+----+------------+----+----+----------------+-----+----------+------+------------+---------+--------+--------------------+---------+
|          0|   0|           0| 769|   0|               0|    0|         0|     0|           0|        0|       0|                   0|        0|
+-----------+----+------------+----+----+----------------+-----+----------+------+------------+---------+--------+--------------------+---------+



###### ELIMINACION DE DATOS NULOS

In [14]:
df_vacunacion_limpio = df_vacunacion.dropna(subset=["EDAD"])

In [27]:
nulos_por_columna_df_limpio = df_vacunacion_limpio.select([sum(col(c).isNull().cast("int")).alias(c) for c in df_vacunacion_limpio.columns])

In [28]:
nulos_por_columna_df_limpio.show()

+-----------+----+------------+----+----+----------------+-----+----------+------+------------+---------+--------+--------------------+---------+
|FECHA_CORTE|UUID|GRUPO_RIESGO|EDAD|SEXO|FECHA_VACUNACION|DOSIS|FABRICANTE|DIRESA|DEPARTAMENTO|PROVINCIA|DISTRITO|CLASIFICACION_VACUNA|TIPO_EDAD|
+-----------+----+------------+----+----+----------------+-----+----------+------+------------+---------+--------+--------------------+---------+
|          0|   0|           0|   0|   0|               0|    0|         0|     0|           0|        0|       0|                   0|        0|
+-----------+----+------------+----+----+----------------+-----+----------+------+------------+---------+--------+--------------------+---------+



###### FORMATEO DEL CAMPO FECHA_VACUNACION

In [15]:
df_vacunacion_limpio = df_vacunacion_limpio.withColumn("FECHA_VACUNACION", to_date(col("FECHA_VACUNACION"), "yyyyMMdd"))

###### DATAFRAME REGISTRO VACUNACION LIMPIO Y FORMATEADO

In [16]:
df_vacunacion_limpio.limit(1000).toPandas()

,FECHA_CORTE,UUID,GRUPO_RIESGO,EDAD,SEXO,FECHA_VACUNACION,DOSIS,FABRICANTE,DIRESA,DEPARTAMENTO,PROVINCIA,DISTRITO,CLASIFICACION_VACUNA,TIPO_EDAD
0,20231210,32633894,PERSONAS DE 18 A 19 AÑOS,19,MASCULINO,2021-09-11,1,PFIZER,TACNA,TACNA,TACNA,CORONEL GREGORIO ALBARRACIN LANCHIP,M,A
1,20231210,20142491,PERSONAS DE 5 A 7 AÑOS,6,MASCULINO,2022-12-08,3,PFIZER,LIMA DIRIS ESTE,LIMA,LIMA,EL AGUSTINO,M,A
2,20231210,33351266,PERSONAS DE 5 A 7 AÑOS,6,MASCULINO,2022-09-26,3,PFIZER,UCAYALI,UCAYALI,CORONEL PORTILLO,CALLERIA,M,A
3,20231210,30522825,PERSONAS DE 5 A 7 AÑOS,5,MASCULINO,2022-04-17,2,PFIZER,CAJAMARCA,CAJAMARCA,CUTERVO,SANTO TOMAS,M,A
4,20231210,35534099,PERSONAS DE 5 A 7 AÑOS,5,MASCULINO,2022-02-17,1,PFIZER,JUNIN,JUNIN,CHUPACA,CHUPACA,M,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,20231210,35675052,PERSONAS DE 8 A 9 AÑOS,8,MASCULINO,2022-03-30,2,PFIZER,AREQUIPA,AREQUIPA,AREQUIPA,CERRO COLORADO,M,A
996,20231210,20563054,PERSONAS DE 8 A 9 AÑOS,8,MASCULINO,2022-03-09,1,PFIZER,LA LIBERTAD,LA LIBERTAD,PACASMAYO,GUADALUPE,M,A
997,20231210,21755006,PERSONAS DE 5 A 7 AÑOS,7,FEMENINO,2022-05-14,2,PFIZER,LAMBAYEQUE,LAMBAYEQUE,FERREÑAFE,INCAHUASI,M,A
998,20231210,33025021,PERSONAS DE 8 A 9 AÑOS,8,FEMENINO,2022-02-23,2,PFIZER,LIMA PROVINCIAS,LIMA,CAÑETE,IMPERIAL,M,A


##### FILTRO Y LIMPIEZA DE DATOS MASIVOS CASOS POSITIVOS

###### DATAFRAME DF_CASOS ANTES DE LA LIMPIEZA Y FORMATEO

In [16]:
df_casos

,FECHA_CORTE,DEPARTAMENTO,PROVINCIA,DISTRITO,METODODX,EDAD,SEXO,FECHA_RESULTADO,UBIGEO,id_persona
0,20211213,LIMA,LIMA,SAN MARTIN DE PORRES,PR,25.0,MASCULINO,20201217.0,150135,24662153
1,20211213,ICA,PISCO,PISCO,PR,20.0,FEMENINO,20200822.0,110501,24662175
2,20211213,HUANUCO,HUANUCO,HUANUCO,PR,22.0,FEMENINO,20200729.0,100101,24662197
3,20211213,ANCASH,SANTA,SANTA,AG,18.0,FEMENINO,20210630.0,021808,24662204
4,20211213,ANCASH,SANTA,NUEVO CHIMBOTE,AG,17.0,MASCULINO,20210404.0,021809,24662207
...,...,...,...,...,...,...,...,...,...,...
2256791,20211213,LIMA,HUAURA,HUACHO,PCR,46.0,MASCULINO,20210206.0,150801,NaN
2256792,20211213,LIMA,LIMA,SAN MARTIN DE PORRES,AG,34.0,MASCULINO,20210327.0,150135,NaN
2256793,20211213,LIMA,LIMA,SAN MARTIN DE PORRES,PCR,34.0,MASCULINO,20210405.0,150135,NaN
2256794,20211213,CALLAO,CALLAO,BELLAVISTA,PCR,28.0,MASCULINO,20210407.0,070102,NaN


###### VERIFICACION DE CAMPOS NULOS

In [32]:
df_casos.isna().sum()

FECHA_CORTE             0
DEPARTAMENTO            0
PROVINCIA               0
DISTRITO                0
METODODX                0
EDAD                  347
SEXO                    1
FECHA_RESULTADO      2023
UBIGEO             110689
id_persona          39116
dtype: int64

###### ELIMINACION DE DATOS NULOS

In [17]:
df_casos_limpio = df_casos.dropna(subset=["EDAD", "SEXO", "FECHA_RESULTADO", "UBIGEO", "id_persona"])

###### VERIFICACION DE DATOS LIMPIOS CASOS POSITIVOS

In [34]:
num_registros_eliminados = len(df_casos) - len(df_casos_limpio)
print(f"Se han eliminado {num_registros_eliminados} registros")

Se han eliminado 150809 registros


In [18]:
df_casos_limpio.isna().sum()

FECHA_CORTE        0
DEPARTAMENTO       0
PROVINCIA          0
DISTRITO           0
METODODX           0
EDAD               0
SEXO               0
FECHA_RESULTADO    0
UBIGEO             0
id_persona         0
dtype: int64

###### FORMATEO DEL CAMPO FECHA_RESULTADO

In [19]:
df_casos_limpio.loc[:, 'FECHA_RESULTADO'] = pd.to_datetime(
    df_casos_limpio['FECHA_RESULTADO'], 
    format='%Y%m%d', 
    errors='coerce'
)

###### CAMBIO DE FORMATO EDAD

In [20]:
df_casos_limpio['EDAD'] = pd.to_numeric(df_casos_limpio['EDAD'], errors='coerce').astype('Int64')

C:\Users\Administrador\AppData\Local\Temp\ipykernel_1236\2212131720.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_casos_limpio['EDAD'] = pd.to_numeric(df_casos_limpio['EDAD'], errors='coerce').astype('Int64')


###### DATAFRAME CASOS POSITIVOS LIMPIO Y FORMATEADO

In [21]:
df_casos_limpio

,FECHA_CORTE,DEPARTAMENTO,PROVINCIA,DISTRITO,METODODX,EDAD,SEXO,FECHA_RESULTADO,UBIGEO,id_persona
0,20211213,LIMA,LIMA,SAN MARTIN DE PORRES,PR,25,MASCULINO,2020-12-17,150135,24662153
1,20211213,ICA,PISCO,PISCO,PR,20,FEMENINO,2020-08-22,110501,24662175
2,20211213,HUANUCO,HUANUCO,HUANUCO,PR,22,FEMENINO,2020-07-29,100101,24662197
3,20211213,ANCASH,SANTA,SANTA,AG,18,FEMENINO,2021-06-30,021808,24662204
4,20211213,ANCASH,SANTA,NUEVO CHIMBOTE,AG,17,MASCULINO,2021-04-04,021809,24662207
...,...,...,...,...,...,...,...,...,...,...
2217753,20211213,PIURA,PIURA,PIURA,AG,33,FEMENINO,2021-11-22,200101,18719016
2217776,20211213,PIURA,SULLANA,BELLAVISTA,AG,15,FEMENINO,2021-04-23,200602,19248215
2217777,20211213,LIMA,LIMA,SAN JUAN DE LURIGANCHO,PR,25,MASCULINO,2020-08-16,150132,19248299
2217822,20211213,ICA,CHINCHA,EL CARMEN,PR,32,FEMENINO,2020-07-23,110205,19420152


##### FILTRO Y LIMPIEZA DE DATOS MASIVOS FALLECIDOS COVID

###### DATAFRAME DF_FALLECIDOS ANTES DE LA LIMPIEZA Y FORMATEO

In [21]:
df_fallecidos

,FECHA_CORTE,FECHA_FALLECIMIENTO,EDAD_DECLARADA,SEXO,CLASIFICACION_DEF,DEPARTAMENTO,PROVINCIA,DISTRITO,UBIGEO,id_persona
0,20231114,20210611,21,MASCULINO,Criterio SINADEF,LAMBAYEQUE,CHICLAYO,CHICLAYO,140101,24833991
1,20231114,20210317,45,MASCULINO,Criterio serolÃ³gico,PIURA,SULLANA,SULLANA,200601,24761117
2,20231114,20210602,62,FEMENINO,Criterio virolÃ³gico,ICA,PISCO,SAN CLEMENTE,110507,24767070
3,20231114,20210703,75,MASCULINO,Criterio virolÃ³gico,AREQUIPA,AREQUIPA,MIRAFLORES,040110,24751741
4,20231114,20210506,66,FEMENINO,Criterio virolÃ³gico,LIMA,LIMA,SAN JUAN DE LURIGANCHO,150132,24671820
...,...,...,...,...,...,...,...,...,...,...
201897,20231114,20210222,76,FEMENINO,Criterio SINADEF,LIMA,LIMA,SAN JUAN DE LURIGANCHO,150132,NaN
201898,20231114,20200508,50,MASCULINO,Criterio SINADEF,JUNIN,HUANCAYO,HUANCAYO,120101,NaN
201899,20231114,20200703,56,FEMENINO,Criterio investigaciÃ³n EpidemiolÃ³gica,LAMBAYEQUE,CHICLAYO,CHICLAYO,140101,NaN
201900,20231114,20210412,31,FEMENINO,Criterio virolÃ³gico,LIMA,LIMA,SAN JUAN DE MIRAFLORES,150133,NaN


###### VERIFICACION DE CAMPOS NULOS

In [22]:
df_fallecidos.isna().sum()

FECHA_CORTE               0
FECHA_FALLECIMIENTO       0
EDAD_DECLARADA            0
SEXO                      0
CLASIFICACION_DEF         0
DEPARTAMENTO              0
PROVINCIA                 5
DISTRITO                  5
UBIGEO                    0
id_persona             3390
dtype: int64

###### ELIMINACION DE DATOS NULOS

In [23]:
df_fallecidos_limpio = df_fallecidos.dropna(subset=["id_persona", "PROVINCIA"])

###### VERIFICACION DE DATOS LIMPIOS CASOS POSITIVOS

In [26]:
num_registros_eliminados_fallecidos = len(df_fallecidos) - len(df_fallecidos_limpio)
print(f"Se han eliminado {num_registros_eliminados_fallecidos} registros")

Se han eliminado 3393 registros


In [24]:
df_fallecidos_limpio.isna().sum()

FECHA_CORTE            0
FECHA_FALLECIMIENTO    0
EDAD_DECLARADA         0
SEXO                   0
CLASIFICACION_DEF      0
DEPARTAMENTO           0
PROVINCIA              0
DISTRITO               0
UBIGEO                 0
id_persona             0
dtype: int64

###### CAMBIO DE FORMATO FECHA_FALLECIMIENTO

In [25]:
df_fallecidos_limpio.loc[:, "FECHA_FALLECIMIENTO"] = pd.to_datetime(
    df_fallecidos_limpio["FECHA_FALLECIMIENTO"],
    format = "%Y%m%d",
    errors = "coerce"
)

###### CORRECCION DE CODIFICACION DEL CAMPO CLASIFICACION DEF

In [26]:
df_fallecidos_limpio['CLASIFICACION_DEF'] = df_fallecidos_limpio['CLASIFICACION_DEF'].str.encode('latin-1').str.decode('utf-8')

C:\Users\Administrador\AppData\Local\Temp\ipykernel_1236\3445994769.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fallecidos_limpio['CLASIFICACION_DEF'] = df_fallecidos_limpio['CLASIFICACION_DEF'].str.encode('latin-1').str.decode('utf-8')


###### DATAFRAME DF_FALLECIDOS LIMPIO Y FORMATEADO

In [29]:
df_fallecidos_limpio

,FECHA_CORTE,FECHA_FALLECIMIENTO,EDAD_DECLARADA,SEXO,CLASIFICACION_DEF,DEPARTAMENTO,PROVINCIA,DISTRITO,UBIGEO,id_persona
0,20231114,2021-06-11,21,MASCULINO,Criterio SINADEF,LAMBAYEQUE,CHICLAYO,CHICLAYO,140101,24833991
1,20231114,2021-03-17,45,MASCULINO,Criterio serológico,PIURA,SULLANA,SULLANA,200601,24761117
2,20231114,2021-06-02,62,FEMENINO,Criterio virológico,ICA,PISCO,SAN CLEMENTE,110507,24767070
3,20231114,2021-07-03,75,MASCULINO,Criterio virológico,AREQUIPA,AREQUIPA,MIRAFLORES,040110,24751741
4,20231114,2021-05-06,66,FEMENINO,Criterio virológico,LIMA,LIMA,SAN JUAN DE LURIGANCHO,150132,24671820
...,...,...,...,...,...,...,...,...,...,...
198603,20231114,2021-05-02,79,FEMENINO,Criterio virológico,JUNIN,CONCEPCION,CONCEPCION,120201,19082386
198619,20231114,2021-05-19,59,MASCULINO,Criterio radiológico,CAJAMARCA,CAJAMARCA,CAJAMARCA,060101,19076214
198620,20231114,2021-05-13,80,MASCULINO,Criterio virológico,LIMA,LIMA,SAN MARTIN DE PORRES,150135,19076321
198621,20231114,2021-04-24,55,MASCULINO,Criterio virológico,LAMBAYEQUE,CHICLAYO,CHICLAYO,140101,18993408


##### FILTRO Y LIMPIEZA DE DATOS MASIVOS PERSONAS

###### LECTURA DEL ARCHIVO CSV TB_DEPARTAMENTO

In [125]:
schema_conversion_departamento = StructType([
    StructField("COD_DEPARTAMENTO", StringType(), True),
    StructField("NOM_DEPARTAMENTO", StringType(), True)
])

In [126]:
df_departamento = spark.read.csv("../FINAL-CSV'S/TB_DEPARTAMENTO.csv", schema=schema_conversion_departamento)

###### LECTURA DEL ARCHIVO CSV TB_PROVINCIA

In [124]:
schema_conversion_provincia = StructType([
    StructField("COD_PROVINCIA", StringType(), True),
    StructField("NOM_PROVINCIA", StringType(), True),
    StructField("COD_DEPARTAMENTO", StringType(), True)
])

In [127]:
df_provincia = spark.read.csv("../FINAL-CSV'S/TB_PROVINCIA.csv", schema=schema_conversion_provincia)

###### LECTURA DEL ARCHIVO CSV TB_DISTRITO

In [128]:
schema_conversion_distrito = StructType([
    StructField("COD_UBIGEO", StringType(), True),
    StructField("NOM_DISTRITO", StringType(), True),
    StructField("COD_PROVINCIA", StringType(), True)
])

In [129]:
df_distrito = spark.read.csv("../FINAL-CSV'S/TB_DISTRITO.csv", schema=schema_conversion_distrito)

###### CREACION DE DF_CASOS_PERSONAS

In [33]:
df_casos_personas = pd.DataFrame(
    {
        "COD_UUID": df_casos_limpio["id_persona"],
        "EDAD": df_casos_limpio["EDAD"],
        "SEXO": df_casos_limpio["SEXO"],
        "COD_UBIGEO": df_casos_limpio["UBIGEO"]
    }
)

###### CONVERSION A ARCHIVO CSV

In [53]:
df_casos_personas.to_csv("../CSV'S-PARCIALES/TB_CASOS_PERSONAS.csv", index=False)

###### LECTURA DEL CSV

In [34]:
schema_conversion = StructType([
    StructField("COD_UUID", StringType(), True),
    StructField("EDAD", IntegerType(), True),
    StructField("SEXO", StringType(), True),
    StructField("COD_UBIGEO", StringType(), True)
])

In [35]:
df_spark_casos_personas = spark.read.csv("../CSV'S-PARCIALES/TB_CASOS_PERSONAS.csv", schema=schema_conversion)

###### NUMERO DE REGISTROS DEL DF_SPARK_CASOS_PERSONAS

In [134]:
num_casos_personas = df_spark_casos_personas.count()
print(f"El numero de registros del df_casos_personas asciende a: {num_casos_personas}")

El numero de registros del df_casos_personas asciende a: 2105988


###### CREACION DE DF_FALLECIDOS_PERSONAS

In [37]:
df_fallecidos_personas = pd.DataFrame(
    {
        "COD_UUID": df_fallecidos_limpio["id_persona"],
        "EDAD": df_fallecidos_limpio["EDAD_DECLARADA"],
        "SEXO": df_fallecidos_limpio["SEXO"],
        "COD_UBIGEO": df_fallecidos_limpio["UBIGEO"]
    }
)

###### CONVERSION A ARCHIVO CSV

In [58]:
df_fallecidos_personas.to_csv("../CSV'S-PARCIALES/TB_FALLECIDOS_PERSONAS.csv", index=False)

###### LECTURA DEL CSV

In [38]:
df_spark_fallecidos_personas = spark.read.csv("../CSV'S-PARCIALES/TB_FALLECIDOS_PERSONAS.csv", schema=schema_conversion)

###### NUMERO DE REGISTROS DEL DF_SPARK_FALLECIDOS_PERSONAS

In [135]:
num_fallecidos_personas = df_spark_fallecidos_personas.count()
print(f"El numero de registro del df_fallecidos_personas asciende a: {num_fallecidos_personas}")

El numero de registro del df_fallecidos_personas asciende a: 198510


###### CREACION DE DF_VACUNACION_PERSONAS

In [130]:
dicc_vacunacion_personas = df_vacunacion_limpio.select(
    col("UUID").alias("COD_UUID"),
    col("EDAD"),
    col("SEXO"),
    col("DEPARTAMENTO").alias("NOM_DEPARTAMENTO"),
    col("PROVINCIA").alias("NOM_PROVINCIA"),
    col("DISTRITO").alias("NOM_DISTRITO")
)

###### ELIMINACION DE DATOS DUPLICADOS POR EL CAMPO UUID

In [131]:
dicc_vacunacion_personas = dicc_vacunacion_personas.dropDuplicates(subset=["COD_UUID"])

###### NUMERO DE DATOS DEL DATAFRAME

In [136]:
num_vacunacion_personas = dicc_vacunacion_personas.count()
print(f"El numero de registros del df_vacunacion_personas asciende a: {num_vacunacion_personas}")

El numero de registros del df_vacunacion_personas asciende a: 30932728


###### VISUALIZACION FINAL DEL DATFRAME DF_VACUNACION_PERSONAS

In [132]:
dicc_vacunacion_personas.limit(100).toPandas()

,COD_UUID,EDAD,SEXO,NOM_DEPARTAMENTO,NOM_PROVINCIA,NOM_DISTRITO
0,10000172,28,MASCULINO,LORETO,MAYNAS,SAN JUAN BAUTISTA
1,10000304,4,FEMENINO,LORETO,DATEM DEL MARAÑON,PASTAZA
2,10000454,31,FEMENINO,ANCASH,SANTA,CHIMBOTE
3,10000472,39,MASCULINO,CUSCO,CHUMBIVILCAS,LLUSCO
4,10000508,32,FEMENINO,TACNA,TACNA,CALANA
...,...,...,...,...,...,...
95,10010944,5,FEMENINO,LA LIBERTAD,BOLIVAR,BOLIVAR
96,10011000,9,MASCULINO,CALLAO,CALLAO,MI PERU
97,10011033,5,FEMENINO,JUNIN,YAULI,LA OROYA
98,10011169,38,FEMENINO,LIMA,LIMA,PACHACAMAC


###### FILTRO Y JOINS PARA LA CREACION DEL DF_VACUNACION_PERSONAS

In [42]:
df_vacunacion_personas = (
    dicc_vacunacion_personas
    .join(df_distrito, dicc_vacunacion_personas.NOM_DISTRITO == df_distrito.NOM_DISTRITO, "inner")
    .join(df_provincia, df_distrito.COD_PROVINCIA == df_provincia.COD_PROVINCIA, "inner")
    .join(df_departamento, df_provincia.COD_DEPARTAMENTO == df_departamento.COD_DEPARTAMENTO, "inner")
    .select(
        dicc_vacunacion_personas.COD_UUID,
        dicc_vacunacion_personas.EDAD,
        dicc_vacunacion_personas.SEXO,
        df_distrito.COD_UBIGEO
    )
).filter(
    (df_distrito.NOM_DISTRITO == dicc_vacunacion_personas.NOM_DISTRITO) &
    (df_provincia.NOM_PROVINCIA == dicc_vacunacion_personas.NOM_PROVINCIA) &
    (df_departamento.NOM_DEPARTAMENTO == dicc_vacunacion_personas.NOM_DEPARTAMENTO)
)

###### VERIFICACION DE DATOS NULOS POR EL CAMPO COD_UBIGEO

In [137]:
valores_nulos_vacunacion = df_vacunacion_personas.filter(col("COD_UBIGEO").isNull()).count()
print(f"El numero de valores nulos del df_vacunacion_personas despues del join: {valores_nulos_vacunacion}")

El numero de valores nulos del df_vacunacion_personas despues del join: 0


###### VISUALIZACION FINAL DEL DATFRAME DF_VACUNACION_PERSONAS

In [48]:
df_vacunacion_personas.limit(10).toPandas()

,UUID,EDAD,SEXO,COD_UBIGEO
0,10000172,28,MASCULINO,160113
1,10000304,4,FEMENINO,160705
2,10000454,31,FEMENINO,021801
3,10000472,39,MASCULINO,080706
4,10000508,32,FEMENINO,230103
5,10000528,32,MASCULINO,020101
6,10000591,43,FEMENINO,150103
7,10000670,42,FEMENINO,150110
8,10000720,6,MASCULINO,150110
9,10000723,5,FEMENINO,150141


###### CREACION DEL DATAFRAME EN SPARK DF_PERSONAS A TRAVES DE LA CONCATENACON DE LOS DF_VACUNACION_PERSONAS, DF_SPARK_CASOS_PERSONAS Y DF_SPARK_FALLECIDOS_PERSONAS

In [43]:
df_personas = df_vacunacion_personas.unionAll(df_spark_casos_personas).unionAll(df_spark_fallecidos_personas)

###### ELIMINACION DE REGISTROS DUPLICADOS POR EL CAMPO UUID

In [44]:
df_personas = df_personas.dropDuplicates(subset=["COD_UUID"])

###### NUMERO DE REGISTROS DEL DF_PERSONAS

In [138]:
num_personas = df_personas.count()
print(f"El numero de registros final de personas asciende a: {num_personas}")

El numero de registros final de personas asciende a: 31198482


###### VISUALIZACION FINAL DEL DATFRAME DF_PERSONAS

In [67]:
df_personas.limit(100).toPandas()

,COD_UUID,EDAD,SEXO,COD_UBIGEO
0,10000007,24,FEMENINO,110505
1,10000018,5,FEMENINO,021809
2,10000136,5,FEMENINO,140105
3,1000015,62,MASCULINO,200101
4,10000154,26,FEMENINO,130107
...,...,...,...,...
95,10001123,69,FEMENINO,150142
96,1000113,59,MASCULINO,221005
97,10001141,55,FEMENINO,150114
98,10001149,80,FEMENINO,150125


#### FILTRO Y LIMPIEZA DE METODO_DETECCION

###### VISUALIZACION DEL DF_CASOS_LIMPIO PARA LA EXTRACCION DE LOS METODOS DE DETECCION

In [45]:
df_casos_limpio

,FECHA_CORTE,DEPARTAMENTO,PROVINCIA,DISTRITO,METODODX,EDAD,SEXO,FECHA_RESULTADO,UBIGEO,id_persona
0,20211213,LIMA,LIMA,SAN MARTIN DE PORRES,PR,25,MASCULINO,2020-12-17,150135,24662153
1,20211213,ICA,PISCO,PISCO,PR,20,FEMENINO,2020-08-22,110501,24662175
2,20211213,HUANUCO,HUANUCO,HUANUCO,PR,22,FEMENINO,2020-07-29,100101,24662197
3,20211213,ANCASH,SANTA,SANTA,AG,18,FEMENINO,2021-06-30,021808,24662204
4,20211213,ANCASH,SANTA,NUEVO CHIMBOTE,AG,17,MASCULINO,2021-04-04,021809,24662207
...,...,...,...,...,...,...,...,...,...,...
2217753,20211213,PIURA,PIURA,PIURA,AG,33,FEMENINO,2021-11-22,200101,18719016
2217776,20211213,PIURA,SULLANA,BELLAVISTA,AG,15,FEMENINO,2021-04-23,200602,19248215
2217777,20211213,LIMA,LIMA,SAN JUAN DE LURIGANCHO,PR,25,MASCULINO,2020-08-16,150132,19248299
2217822,20211213,ICA,CHINCHA,EL CARMEN,PR,32,FEMENINO,2020-07-23,110205,19420152


###### TIPOS DE METODOS DE DETECCION

In [46]:
metodos_unicos = pd.Series(df_casos_limpio["METODODX"].unique())
print(metodos_unicos)

0     PR
1     AG
2    PCR
dtype: object


###### DATAFRAME DF_METODO LIMPIO

In [47]:
df_metodo = pd.DataFrame(
    {
      "COD_METODO": range(1, len(metodos_unicos) + 1),
      "TIPO_METODO" : metodos_unicos
    }
)

###### DATAFRAME DF_METODO LIMPIO Y FORMATEADO

In [48]:
df_metodo

,COD_METODO,TIPO_METODO
0,1,PR
1,2,AG
2,3,PCR


#### CREACION DEL DF_CASOS_POSITIVOS A PARTIR DEL JOIN METODODX Y METODO_ID

###### VISUALIZACION DEL DF_CASOS_LIMPIO PARA LA APLICACION DE JOINS

In [49]:
df_casos_limpio

,FECHA_CORTE,DEPARTAMENTO,PROVINCIA,DISTRITO,METODODX,EDAD,SEXO,FECHA_RESULTADO,UBIGEO,id_persona
0,20211213,LIMA,LIMA,SAN MARTIN DE PORRES,PR,25,MASCULINO,2020-12-17,150135,24662153
1,20211213,ICA,PISCO,PISCO,PR,20,FEMENINO,2020-08-22,110501,24662175
2,20211213,HUANUCO,HUANUCO,HUANUCO,PR,22,FEMENINO,2020-07-29,100101,24662197
3,20211213,ANCASH,SANTA,SANTA,AG,18,FEMENINO,2021-06-30,021808,24662204
4,20211213,ANCASH,SANTA,NUEVO CHIMBOTE,AG,17,MASCULINO,2021-04-04,021809,24662207
...,...,...,...,...,...,...,...,...,...,...
2217753,20211213,PIURA,PIURA,PIURA,AG,33,FEMENINO,2021-11-22,200101,18719016
2217776,20211213,PIURA,SULLANA,BELLAVISTA,AG,15,FEMENINO,2021-04-23,200602,19248215
2217777,20211213,LIMA,LIMA,SAN JUAN DE LURIGANCHO,PR,25,MASCULINO,2020-08-16,150132,19248299
2217822,20211213,ICA,CHINCHA,EL CARMEN,PR,32,FEMENINO,2020-07-23,110205,19420152


###### FILTRO CRUZADO

In [50]:
dicc_casos_metodo = pd.merge(df_casos_limpio, df_metodo, how="left", left_on="METODODX", right_on="TIPO_METODO")

###### SELECCION DE CAMPOS

In [51]:
dicc_casos_metodo = dicc_casos_metodo[["id_persona", "METODODX", "COD_METODO", "FECHA_RESULTADO"]]

###### VISUALIZACION DEL DATAFRAME DICC_CASOS_METODO

In [52]:
dicc_casos_metodo

,id_persona,METODODX,COD_METODO,FECHA_RESULTADO
0,24662153,PR,1,2020-12-17
1,24662175,PR,1,2020-08-22
2,24662197,PR,1,2020-07-29
3,24662204,AG,2,2021-06-30
4,24662207,AG,2,2021-04-04
...,...,...,...,...
2105982,18719016,AG,2,2021-11-22
2105983,19248215,AG,2,2021-04-23
2105984,19248299,PR,1,2020-08-16
2105985,19420152,PR,1,2020-07-23


###### CREACION DEL DF_CASOS_POSITIVOS DESPUES DE TODOS LOS FILTROS Y LIMPIEZA

In [53]:
df_casos_positivo = pd.DataFrame(
    {
        "COD_CASO": range(1, len(dicc_casos_metodo) + 1),
        "COD_UUID": dicc_casos_metodo["id_persona"],
        "COD_METODO": dicc_casos_metodo["COD_METODO"],
        "FECHA_RESULTADO": dicc_casos_metodo["FECHA_RESULTADO"]        
    }
)

###### VISUALIZACION DEL DF_CASOS_POSITIVO

In [54]:
df_casos_positivo

,COD_CASO,COD_UUID,COD_METODO,FECHA_RESULTADO
0,1,24662153,1,2020-12-17
1,2,24662175,1,2020-08-22
2,3,24662197,1,2020-07-29
3,4,24662204,2,2021-06-30
4,5,24662207,2,2021-04-04
...,...,...,...,...
2105982,2105983,18719016,2,2021-11-22
2105983,2105984,19248215,2,2021-04-23
2105984,2105985,19248299,1,2020-08-16
2105985,2105986,19420152,1,2020-07-23


#### FILTRO Y LIMPIEZA DE CLASIFICACION DEFUNCION

###### VISUALIZACION DEL DF_FALLECIDOS_LIMPIO PARA LA EXTRACCION DE LAS CLASIFICACIONES DE DEFUNCION

In [55]:
df_fallecidos_limpio

,FECHA_CORTE,FECHA_FALLECIMIENTO,EDAD_DECLARADA,SEXO,CLASIFICACION_DEF,DEPARTAMENTO,PROVINCIA,DISTRITO,UBIGEO,id_persona
0,20231114,2021-06-11,21,MASCULINO,Criterio SINADEF,LAMBAYEQUE,CHICLAYO,CHICLAYO,140101,24833991
1,20231114,2021-03-17,45,MASCULINO,Criterio serológico,PIURA,SULLANA,SULLANA,200601,24761117
2,20231114,2021-06-02,62,FEMENINO,Criterio virológico,ICA,PISCO,SAN CLEMENTE,110507,24767070
3,20231114,2021-07-03,75,MASCULINO,Criterio virológico,AREQUIPA,AREQUIPA,MIRAFLORES,040110,24751741
4,20231114,2021-05-06,66,FEMENINO,Criterio virológico,LIMA,LIMA,SAN JUAN DE LURIGANCHO,150132,24671820
...,...,...,...,...,...,...,...,...,...,...
198603,20231114,2021-05-02,79,FEMENINO,Criterio virológico,JUNIN,CONCEPCION,CONCEPCION,120201,19082386
198619,20231114,2021-05-19,59,MASCULINO,Criterio radiológico,CAJAMARCA,CAJAMARCA,CAJAMARCA,060101,19076214
198620,20231114,2021-05-13,80,MASCULINO,Criterio virológico,LIMA,LIMA,SAN MARTIN DE PORRES,150135,19076321
198621,20231114,2021-04-24,55,MASCULINO,Criterio virológico,LAMBAYEQUE,CHICLAYO,CHICLAYO,140101,18993408


###### EXTRACCION DE DATOS UNICOS DEL CAMPO CLASIFICACION_DEF

In [56]:
defuncion_unicos = pd.Series(df_fallecidos_limpio["CLASIFICACION_DEF"].unique())
print(defuncion_unicos)

0                         Criterio SINADEF
1                      Criterio serológico
2                      Criterio virológico
3                         Criterio clínico
4                     Criterio radiológico
5             Criterio nexo epidemiológico
6    Criterio investigación Epidemiológica
dtype: object


###### CREACION DEL DF_CLASIFICACION_DEF

In [57]:
df_clasificacion_def = pd.DataFrame(
    {
        "COD_CLASI_DEFUNCION": range(1, len(defuncion_unicos) + 1),
        "NOM_CLASIFICACION": defuncion_unicos
    }
)

###### DATAFRAME DF_CLASIFICACION_DEF LIMPIO Y FORMATEADO

In [58]:
df_clasificacion_def

,COD_CLASI_DEFUNCION,NOM_CLASIFICACION
0,1,Criterio SINADEF
1,2,Criterio serológico
2,3,Criterio virológico
3,4,Criterio clínico
4,5,Criterio radiológico
5,6,Criterio nexo epidemiológico
6,7,Criterio investigación Epidemiológica


#### CREACION DEL DF_FALLECIDOS_COVID A PARTIR DEL JOIN COD_UUID Y COD_DEFUNCION

###### VISUALIZACION DEL DF_FALLECIDOS_LIMPIO PARA LA APLICACION DE JOINS

In [59]:
df_fallecidos_limpio

,FECHA_CORTE,FECHA_FALLECIMIENTO,EDAD_DECLARADA,SEXO,CLASIFICACION_DEF,DEPARTAMENTO,PROVINCIA,DISTRITO,UBIGEO,id_persona
0,20231114,2021-06-11,21,MASCULINO,Criterio SINADEF,LAMBAYEQUE,CHICLAYO,CHICLAYO,140101,24833991
1,20231114,2021-03-17,45,MASCULINO,Criterio serológico,PIURA,SULLANA,SULLANA,200601,24761117
2,20231114,2021-06-02,62,FEMENINO,Criterio virológico,ICA,PISCO,SAN CLEMENTE,110507,24767070
3,20231114,2021-07-03,75,MASCULINO,Criterio virológico,AREQUIPA,AREQUIPA,MIRAFLORES,040110,24751741
4,20231114,2021-05-06,66,FEMENINO,Criterio virológico,LIMA,LIMA,SAN JUAN DE LURIGANCHO,150132,24671820
...,...,...,...,...,...,...,...,...,...,...
198603,20231114,2021-05-02,79,FEMENINO,Criterio virológico,JUNIN,CONCEPCION,CONCEPCION,120201,19082386
198619,20231114,2021-05-19,59,MASCULINO,Criterio radiológico,CAJAMARCA,CAJAMARCA,CAJAMARCA,060101,19076214
198620,20231114,2021-05-13,80,MASCULINO,Criterio virológico,LIMA,LIMA,SAN MARTIN DE PORRES,150135,19076321
198621,20231114,2021-04-24,55,MASCULINO,Criterio virológico,LAMBAYEQUE,CHICLAYO,CHICLAYO,140101,18993408


###### JOIN POR CAMPO CLASIFICACION_DEF Y COD_DEFUNCION

In [60]:
dicc_fallecidos_join = pd.merge(df_fallecidos_limpio, df_clasificacion_def, how="left", left_on="CLASIFICACION_DEF", right_on="NOM_CLASIFICACION")

###### REDEFINICION DE CAMPOS

In [61]:
dicc_fallecidos_join = dicc_fallecidos_join[["id_persona", "COD_CLASI_DEFUNCION", "FECHA_FALLECIMIENTO"]]

###### VISUALIZACION DEL DATAFRAME DICC_FALLECIDOS_JOIN

In [62]:
dicc_fallecidos_join

,id_persona,COD_CLASI_DEFUNCION,FECHA_FALLECIMIENTO
0,24833991,1,2021-06-11
1,24761117,2,2021-03-17
2,24767070,3,2021-06-02
3,24751741,3,2021-07-03
4,24671820,3,2021-05-06
...,...,...,...
198504,19082386,3,2021-05-02
198505,19076214,5,2021-05-19
198506,19076321,3,2021-05-13
198507,18993408,3,2021-04-24


###### CREACION DEL DF_FALLECIDOS_COVID DESPUES DE TODOS LOS FILTROS Y LIMPIEZA

In [63]:
df_fallecidos_covid = pd.DataFrame(
    {
        "COD_DECESO": range(1, len(dicc_fallecidos_join) + 1),
        "COD_UUID": dicc_fallecidos_join["id_persona"],
        "COD_CLASI_DEFUNCION": dicc_fallecidos_join["COD_CLASI_DEFUNCION"],
        "FECHA_FALLECIMIENTO": dicc_fallecidos_join["FECHA_FALLECIMIENTO"]
    }
)

###### VISUALIZACION DEL DF_FALLECIDOS_COVID

In [64]:
df_fallecidos_covid

,COD_DECESO,COD_UUID,COD_CLASI_DEFUNCION,FECHA_FALLECIMIENTO
0,1,24833991,1,2021-06-11
1,2,24761117,2,2021-03-17
2,3,24767070,3,2021-06-02
3,4,24751741,3,2021-07-03
4,5,24671820,3,2021-05-06
...,...,...,...,...
198504,198505,19082386,3,2021-05-02
198505,198506,19076214,5,2021-05-19
198506,198507,19076321,3,2021-05-13
198507,198508,18993408,3,2021-04-24


#### FILTRO Y LIMPIEZA DE DIRESA

###### VISUALIZACION DEL DF_VACUNACION_LIMPIO PARA LA EXTRACCION DE LAS DIRESA

In [65]:
df_vacunacion_limpio.limit(100).toPandas()

,FECHA_CORTE,UUID,GRUPO_RIESGO,EDAD,SEXO,FECHA_VACUNACION,DOSIS,FABRICANTE,DIRESA,DEPARTAMENTO,PROVINCIA,DISTRITO,CLASIFICACION_VACUNA,TIPO_EDAD
0,20231210,32633894,PERSONAS DE 18 A 19 AÑOS,19,MASCULINO,2021-09-11,1,PFIZER,TACNA,TACNA,TACNA,CORONEL GREGORIO ALBARRACIN LANCHIP,M,A
1,20231210,20142491,PERSONAS DE 5 A 7 AÑOS,6,MASCULINO,2022-12-08,3,PFIZER,LIMA DIRIS ESTE,LIMA,LIMA,EL AGUSTINO,M,A
2,20231210,33351266,PERSONAS DE 5 A 7 AÑOS,6,MASCULINO,2022-09-26,3,PFIZER,UCAYALI,UCAYALI,CORONEL PORTILLO,CALLERIA,M,A
3,20231210,30522825,PERSONAS DE 5 A 7 AÑOS,5,MASCULINO,2022-04-17,2,PFIZER,CAJAMARCA,CAJAMARCA,CUTERVO,SANTO TOMAS,M,A
4,20231210,35534099,PERSONAS DE 5 A 7 AÑOS,5,MASCULINO,2022-02-17,1,PFIZER,JUNIN,JUNIN,CHUPACA,CHUPACA,M,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,20231210,86014,"60 AÑOS A MÁS, ADULTO MAYOR",69,FEMENINO,2021-08-08,1,SINOPHARM,UCAYALI,UCAYALI,CORONEL PORTILLO,CALLERIA,M,A
96,20231210,104239,"60 AÑOS A MÁS, ADULTO MAYOR",61,MASCULINO,2023-09-01,5,MODERNA,UCAYALI,UCAYALI,CORONEL PORTILLO,CALLERIA,B,A
97,20231210,58879,PERSONAS DE 50 A 59 AÑOS,54,FEMENINO,2022-09-12,4,MODERNA,UCAYALI,UCAYALI,CORONEL PORTILLO,CALLERIA,M,A
98,20231210,24153353,PERSONAS DE 50 A 59 AÑOS,55,MASCULINO,2022-11-09,4,MODERNA,HUANUCO,HUANUCO,LEONCIO PRADO,RUPA-RUPA,M,A


###### EXTRACCION DE DATOS UNICOS DEL CAMPO DIRESA

In [ ]:
diresa_unicos = df_vacunacion_limpio.select("DIRESA").distinct()

###### CONVERSION A DATAFRAME DE PANDAS

In [69]:
dicc_diresa =  diresa_unicos.toPandas()

###### VISUALIZACION DE LOS DATOS UNICOS DIRESA

In [70]:
dicc_diresa

,DIRESA
0,AYACUCHO
1,APURIMAC
2,CALLAO
3,LA LIBERTAD
4,PASCO
5,AMAZONAS
6,UCAYALI
7,LIMA DIRIS NORTE
8,TUMBES
9,JUNIN


###### CREACION DEL DF_DIRESA DESPUES DEL FILTRO Y LIMPIEZA

In [71]:
df_diresa = pd.DataFrame(
    {
        "COD_DIRESA": range(1, len(dicc_diresa) + 1),
        "CENTRO_DIRESA": dicc_diresa["DIRESA"]
    }
)

###### VISUALIZACION FINAL DEL DF_DIRESA

In [72]:
df_diresa

,COD_DIRESA,CENTRO_DIRESA
0,1,AYACUCHO
1,2,APURIMAC
2,3,CALLAO
3,4,LA LIBERTAD
4,5,PASCO
5,6,AMAZONAS
6,7,UCAYALI
7,8,LIMA DIRIS NORTE
8,9,TUMBES
9,10,JUNIN


#### FILTRO Y LIMPIEZA DE FABRICANTE

###### VISUALIZACION DEL DF_VACUNACION_LIMPIO PARA LA EXTRACCION DE LOS FABRICANTES DE VACUNAS

In [74]:
df_vacunacion_limpio.limit(100).toPandas()

,FECHA_CORTE,UUID,GRUPO_RIESGO,EDAD,SEXO,FECHA_VACUNACION,DOSIS,FABRICANTE,DIRESA,DEPARTAMENTO,PROVINCIA,DISTRITO,CLASIFICACION_VACUNA,TIPO_EDAD
0,20231210,32633894,PERSONAS DE 18 A 19 AÑOS,19,MASCULINO,2021-09-11,1,PFIZER,TACNA,TACNA,TACNA,CORONEL GREGORIO ALBARRACIN LANCHIP,M,A
1,20231210,20142491,PERSONAS DE 5 A 7 AÑOS,6,MASCULINO,2022-12-08,3,PFIZER,LIMA DIRIS ESTE,LIMA,LIMA,EL AGUSTINO,M,A
2,20231210,33351266,PERSONAS DE 5 A 7 AÑOS,6,MASCULINO,2022-09-26,3,PFIZER,UCAYALI,UCAYALI,CORONEL PORTILLO,CALLERIA,M,A
3,20231210,30522825,PERSONAS DE 5 A 7 AÑOS,5,MASCULINO,2022-04-17,2,PFIZER,CAJAMARCA,CAJAMARCA,CUTERVO,SANTO TOMAS,M,A
4,20231210,35534099,PERSONAS DE 5 A 7 AÑOS,5,MASCULINO,2022-02-17,1,PFIZER,JUNIN,JUNIN,CHUPACA,CHUPACA,M,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,20231210,86014,"60 AÑOS A MÁS, ADULTO MAYOR",69,FEMENINO,2021-08-08,1,SINOPHARM,UCAYALI,UCAYALI,CORONEL PORTILLO,CALLERIA,M,A
96,20231210,104239,"60 AÑOS A MÁS, ADULTO MAYOR",61,MASCULINO,2023-09-01,5,MODERNA,UCAYALI,UCAYALI,CORONEL PORTILLO,CALLERIA,B,A
97,20231210,58879,PERSONAS DE 50 A 59 AÑOS,54,FEMENINO,2022-09-12,4,MODERNA,UCAYALI,UCAYALI,CORONEL PORTILLO,CALLERIA,M,A
98,20231210,24153353,PERSONAS DE 50 A 59 AÑOS,55,MASCULINO,2022-11-09,4,MODERNA,HUANUCO,HUANUCO,LEONCIO PRADO,RUPA-RUPA,M,A


###### EXTRACCION DE DATOS UNICOS DEL CAMPO FABRICANTE

In [82]:
fabricante_unicos = df_vacunacion_limpio.select("FABRICANTE").distinct()

###### CONVERSION A DATAFRAME DE PANDAS

In [83]:
dicc_fabricante = fabricante_unicos.toPandas()

###### VISUALIZACION DE LOS DATOS UNICOS FABRICANTE

In [84]:
dicc_fabricante

,FABRICANTE
0,SINOPHARM
1,MODERNA
2,JANSSEN
3,PFIZER
4,ASTRAZENECA


###### CREACION DEL DF_FABRICANTE DESPUES DEL FILTRO Y LIMPIEZA

In [85]:
df_fabricante = pd.DataFrame(
    {
        "COD_FABRICANTE": range(1, len(dicc_fabricante) + 1),
        "NOM_FABRICANTE": dicc_fabricante["FABRICANTE"]
    }
)

###### VISUALIZACION FINAL DEL DF_FABRICANTE DESPUES DEL FILTRO Y LIMPIEZA

In [86]:
df_fabricante

,COD_FABRICANTE,NOM_FABRICANTE
0,1,SINOPHARM
1,2,MODERNA
2,3,JANSSEN
3,4,PFIZER
4,5,ASTRAZENECA


#### FILTRO Y LIMPIEZA DE GRUPO DE RIESGO

###### VISUALIZACION DEL DF_VACUNACION_LIMPIO PARA LA EXTRACCION DE LOS GRUPOS DE RIESGO

In [88]:
df_vacunacion_limpio.limit(100).toPandas()

,FECHA_CORTE,UUID,GRUPO_RIESGO,EDAD,SEXO,FECHA_VACUNACION,DOSIS,FABRICANTE,DIRESA,DEPARTAMENTO,PROVINCIA,DISTRITO,CLASIFICACION_VACUNA,TIPO_EDAD
0,20231210,32633894,PERSONAS DE 18 A 19 AÑOS,19,MASCULINO,2021-09-11,1,PFIZER,TACNA,TACNA,TACNA,CORONEL GREGORIO ALBARRACIN LANCHIP,M,A
1,20231210,20142491,PERSONAS DE 5 A 7 AÑOS,6,MASCULINO,2022-12-08,3,PFIZER,LIMA DIRIS ESTE,LIMA,LIMA,EL AGUSTINO,M,A
2,20231210,33351266,PERSONAS DE 5 A 7 AÑOS,6,MASCULINO,2022-09-26,3,PFIZER,UCAYALI,UCAYALI,CORONEL PORTILLO,CALLERIA,M,A
3,20231210,30522825,PERSONAS DE 5 A 7 AÑOS,5,MASCULINO,2022-04-17,2,PFIZER,CAJAMARCA,CAJAMARCA,CUTERVO,SANTO TOMAS,M,A
4,20231210,35534099,PERSONAS DE 5 A 7 AÑOS,5,MASCULINO,2022-02-17,1,PFIZER,JUNIN,JUNIN,CHUPACA,CHUPACA,M,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,20231210,86014,"60 AÑOS A MÁS, ADULTO MAYOR",69,FEMENINO,2021-08-08,1,SINOPHARM,UCAYALI,UCAYALI,CORONEL PORTILLO,CALLERIA,M,A
96,20231210,104239,"60 AÑOS A MÁS, ADULTO MAYOR",61,MASCULINO,2023-09-01,5,MODERNA,UCAYALI,UCAYALI,CORONEL PORTILLO,CALLERIA,B,A
97,20231210,58879,PERSONAS DE 50 A 59 AÑOS,54,FEMENINO,2022-09-12,4,MODERNA,UCAYALI,UCAYALI,CORONEL PORTILLO,CALLERIA,M,A
98,20231210,24153353,PERSONAS DE 50 A 59 AÑOS,55,MASCULINO,2022-11-09,4,MODERNA,HUANUCO,HUANUCO,LEONCIO PRADO,RUPA-RUPA,M,A


###### EXTRACCION DE DATOS UNICOS DEL CAMPO GRUPO_RIESGO

In [89]:
grupo_riesgo_unicos = df_vacunacion_limpio.select("GRUPO_RIESGO").distinct()

###### CONVERSION A DATAFRAME DE PANDAS

In [90]:
dicc_grupo_riesgo = grupo_riesgo_unicos.toPandas()

###### VISUALIZACION DE LOS DATOS UNICOS GRUPO_RIESGO

In [91]:
dicc_grupo_riesgo

,GRUPO_RIESGO
0,SERENAZGO
1,SIN GRUPO DE RIESGO
2,6 MESES A 4 AÑOS
3,BRIGADISTAS
4,TRABAJADOR Ó PERSONAL DE LIMPIEZA
5,PERSONAS DE 30 A 39 AÑOS
6,ZONAS ALTOANDINAS
7,PERSONAS DE 70 AÑOS A MÁS
8,PERSONAL QUE LABORA EN INPE
9,PACIENTE ONCOLÓGICO SIN TRATAMIENTO ACTIVO


###### CREACION DEL DF_GRUPO_RIESGO DESPUES DEL FILTRO Y LIMPIEZA

In [92]:
df_grupo_riesgo = pd.DataFrame(
    {
        "COD_GRUPO": range(1, len(dicc_grupo_riesgo) + 1),
        "TIPO_GRUPO": dicc_grupo_riesgo["GRUPO_RIESGO"]
    }
)

###### VISUALIZACION FINAL DEL DF_GRUPO_RIESGO DESPUES DEL FILTRO Y LIMPIEZA

In [93]:
df_grupo_riesgo

,COD_GRUPO,TIPO_GRUPO
0,1,SERENAZGO
1,2,SIN GRUPO DE RIESGO
2,3,6 MESES A 4 AÑOS
3,4,BRIGADISTAS
4,5,TRABAJADOR Ó PERSONAL DE LIMPIEZA
5,6,PERSONAS DE 30 A 39 AÑOS
6,7,ZONAS ALTOANDINAS
7,8,PERSONAS DE 70 AÑOS A MÁS
8,9,PERSONAL QUE LABORA EN INPE
9,10,PACIENTE ONCOLÓGICO SIN TRATAMIENTO ACTIVO


#### CREACION DEL DF_REGISTRO_VACUNACION A PARTIR DEL JOIN COD_DIRESA, COD_GRUPO Y COD_FABRICANTE

###### CREACION DEL SCHEMA DIRESA

In [96]:
schema_diresa = StructType(
    [
        StructField("COD_DIRESA", StringType(), True),
        StructField("CENTRO_DIRESA", StringType(), True)
    ]
)

###### LECTURA DEL CSV TB_DIRESA

In [97]:
df_spark_diresa = spark.read.csv("../FINAL-CSV'S/TB_DIRESA.csv", header=True, schema=schema_diresa)

###### CREACION DEL SCHEMA GRUPO DE RIESGO

In [100]:
schema_grupo_riesgo = StructType(
    [
        StructField("COD_GRUPO", StringType(), True),
        StructField("TIPO_GRUPO", StringType(), True)
    ]
)

###### LECTURA DEL CSV TB_GRUPO_RIESGO

In [101]:
df_spark_grupo_riesgo = spark.read.csv("../FINAL-CSV'S/TB_GRUPO_RIESGO.csv", header=True, schema=schema_grupo_riesgo)

###### CREACION DEL SCHEMA FABRICANTE

In [103]:
schema_fabricante = StructType(
    [
        StructField("COD_FABRICANTE", StringType(), True),
        StructField("NOM_FABRICANTE", StringType(), True)
    ]
)

###### LECTURA DEL CSV TB_FABRICANTE

In [104]:
df_spark_fabricante = spark.read.csv("../FINAL-CSV'S/TB_FABRICANTE.csv", header=True, schema=schema_fabricante)

###### VISUALIZACION DEL DF_VACUNACION_LIMPIO PARA LA APLICACION DE JOINS

In [106]:
df_vacunacion_limpio.limit(100).toPandas()

,FECHA_CORTE,UUID,GRUPO_RIESGO,EDAD,SEXO,FECHA_VACUNACION,DOSIS,FABRICANTE,DIRESA,DEPARTAMENTO,PROVINCIA,DISTRITO,CLASIFICACION_VACUNA,TIPO_EDAD
0,20231210,32633894,PERSONAS DE 18 A 19 AÑOS,19,MASCULINO,2021-09-11,1,PFIZER,TACNA,TACNA,TACNA,CORONEL GREGORIO ALBARRACIN LANCHIP,M,A
1,20231210,20142491,PERSONAS DE 5 A 7 AÑOS,6,MASCULINO,2022-12-08,3,PFIZER,LIMA DIRIS ESTE,LIMA,LIMA,EL AGUSTINO,M,A
2,20231210,33351266,PERSONAS DE 5 A 7 AÑOS,6,MASCULINO,2022-09-26,3,PFIZER,UCAYALI,UCAYALI,CORONEL PORTILLO,CALLERIA,M,A
3,20231210,30522825,PERSONAS DE 5 A 7 AÑOS,5,MASCULINO,2022-04-17,2,PFIZER,CAJAMARCA,CAJAMARCA,CUTERVO,SANTO TOMAS,M,A
4,20231210,35534099,PERSONAS DE 5 A 7 AÑOS,5,MASCULINO,2022-02-17,1,PFIZER,JUNIN,JUNIN,CHUPACA,CHUPACA,M,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,20231210,86014,"60 AÑOS A MÁS, ADULTO MAYOR",69,FEMENINO,2021-08-08,1,SINOPHARM,UCAYALI,UCAYALI,CORONEL PORTILLO,CALLERIA,M,A
96,20231210,104239,"60 AÑOS A MÁS, ADULTO MAYOR",61,MASCULINO,2023-09-01,5,MODERNA,UCAYALI,UCAYALI,CORONEL PORTILLO,CALLERIA,B,A
97,20231210,58879,PERSONAS DE 50 A 59 AÑOS,54,FEMENINO,2022-09-12,4,MODERNA,UCAYALI,UCAYALI,CORONEL PORTILLO,CALLERIA,M,A
98,20231210,24153353,PERSONAS DE 50 A 59 AÑOS,55,MASCULINO,2022-11-09,4,MODERNA,HUANUCO,HUANUCO,LEONCIO PRADO,RUPA-RUPA,M,A


###### FILTRO Y JOINS PARA LA CREACION DEL DF_REGISTRO_VACUNACION

In [111]:
df_registro_vacunacion = (
    df_vacunacion_limpio
    .join(df_spark_diresa, df_vacunacion_limpio.DIRESA == df_spark_diresa.CENTRO_DIRESA, "inner")
    .join(df_spark_grupo_riesgo, df_vacunacion_limpio.GRUPO_RIESGO == df_spark_grupo_riesgo.TIPO_GRUPO, "inner")
    .join(df_spark_fabricante, df_vacunacion_limpio.FABRICANTE == df_spark_fabricante.NOM_FABRICANTE, "inner")
    .select(
        (F.monotonically_increasing_id() + F.lit(1)).alias("COD_VACUNACION"),
        df_vacunacion_limpio.UUID.alias("COD_UUID"),
        df_spark_diresa.COD_DIRESA,
        df_spark_grupo_riesgo.COD_GRUPO,
        df_spark_fabricante.COD_FABRICANTE,
        df_vacunacion_limpio.DOSIS.alias("NUM_DOSIS"),
        df_vacunacion_limpio.FECHA_VACUNACION
    )
).filter(
    (df_spark_diresa.CENTRO_DIRESA == df_vacunacion_limpio.DIRESA) &
    (df_spark_grupo_riesgo.TIPO_GRUPO == df_vacunacion_limpio.GRUPO_RIESGO) &
    (df_spark_fabricante.NOM_FABRICANTE == df_vacunacion_limpio.FABRICANTE)
)

###### NUMERO DE REGISTROS DE VACUNACION

In [123]:
num_registro_vacunacion = df_registro_vacunacion.count()
print(f"El numero de registros final de vacunacion es: {num_registro_vacunacion}")

El numero de registros final de vacunacion es: 91610278


###### VISUALIZACION FINAL DEL DF_REGISTRO_VACUNACION DESPUES DEL JOIN 

In [121]:
df_registro_vacunacion.limit(100).toPandas()

,COD_VACUNACION,COD_UUID,COD_DIRESA,COD_GRUPO,COD_FABRICANTE,NUM_DOSIS,FECHA_VACUNACION
0,1,32633894,23,52,4,1,2021-09-11
1,2,20142491,25,43,4,3,2022-12-08
2,3,33351266,7,43,4,3,2022-09-26
3,4,30522825,20,43,4,2,2022-04-17
4,5,35534099,10,43,4,1,2022-02-17
...,...,...,...,...,...,...,...
95,96,86014,7,40,1,1,2021-08-08
96,97,104239,7,40,2,5,2023-09-01
97,98,58879,7,18,2,4,2022-09-12
98,99,24153353,16,18,2,4,2022-11-09


##### CONVERSION DE ARCHIVOS CSV

###### CONVERSION DEL DF_PERSONAS A ARCHIVO CSV

In [68]:
df_personas.coalesce(1).write.csv("../FINAL-CSV'S/TB_PERSONA.csv", header=True, mode="overwrite")

###### CONVERSION DEL DF_METODO A ARCHIVO CSV

In [64]:
df_metodo.to_csv("../FINAL-CSV'S/TB_METODO.csv", index=False)

###### CONVERSION DEL DF_CASOS_POSITIVO A CSV

In [84]:
df_casos_positivo.to_csv("../FINAL-CSV'S/TB_CASOS_POSITIVO.csv", index=False)

###### CONVERSION DEL DF_CLASIFICACION_DEF A CSV

In [96]:
df_clasificacion_def.to_csv("../FINAL-CSV'S/TB_CLASIFICACION_DEF.csv", index=False)

###### CONVERSION DEL DF_FALLECIDOS_COVID A CSV

In [104]:
df_fallecidos_covid.to_csv("../FINAL-CSV'S/TB_FALLECIDOS_COVID.csv", index=False)

###### CONVERSION DEL DF_DIRESA A CSV

In [73]:
df_diresa.to_csv("../FINAL-CSV'S/TB_DIRESA.csv", index=False)

###### CONVERSION DEL DF_FABRICANTE A CSV

In [87]:
df_fabricante.to_csv("../FINAL-CSV'S/TB_FABRICANTE.csv", index=False)

###### CONVERSION DEL DF_GRUPO_RIESGO A CSV

In [95]:
df_grupo_riesgo.to_csv("../FINAL-CSV'S/TB_GRUPO_RIESGO.csv", index=False)

###### CONVERSION DEL DF_REGISTRO_VACUNACION A CSV

In [122]:
df_registro_vacunacion.coalesce(1).write.csv("../FINAL-CSV'S/TB_REGISTRO_VACUNACION.csv", header=True)